Компьютер загадывает целое число от 1 до 100, и нам его нужно угадать.

Под «угадать», конечно, подразумевается «написать программу, которая угадывает число».

Внизу представлен готовый код, который вы можете запустить и сыграть с компьютером:

In [262]:
import numpy as np
count = 0                            # счетчик попыток
number = np.random.randint(1,101)    # загадали число
print ("Загадано число от 1 до 100")

while True:                        # бесконечный цикл
    predict = int(input())         # предполагаемое число
    count += 1                     # плюсуем попытку
    if number == predict: break    # выход из цикла, если угадали
    elif number > predict: print (f"Угадываемое число больше {predict} ")
    elif number < predict: print (f"Угадываемое число меньше {predict} ")
        
print (f"Вы угадали число {number} за {count} попыток.")

Загадано число от 1 до 100
50
Угадываемое число больше 50 
75
Угадываемое число меньше 75 
62
Угадываемое число больше 62 
67
Угадываемое число меньше 67 
65
Угадываемое число меньше 65 
64
Вы угадали число 64 за 6 попыток.


Автоматизируем процесс:

In [139]:
def game_core_v1(number):
    '''Просто угадываем на random, никак не используя информацию о больше или меньше.
       Функция принимает загаданное число и возвращает число попыток'''
    count = 0
    while True:
        count+=1
        predict = np.random.randint(1,101) # предполагаемое число
        if number == predict: 
            return(count) # выход из цикла, если угадали

Повторим 1000 раз, чтоб понять, насколько быстро в среднем работает наше решение.  
Для этого сразу завернем игру в функцию, чтобы её потом было проще менять и к ней обращаться.

In [140]:
import numpy as np
def score_game(game_core):
    '''Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает число'''
    count_ls = []
    np.random.seed(1)  # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    random_array = np.random.randint(1,101, size=(1000))
    for number in random_array:
        count_ls.append(game_core(number))
    score = int(np.mean(count_ls))
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток")
    return(score)

In [141]:
# запускаем
score_game(game_core_v1)

Ваш алгоритм угадывает число в среднем за 101 попыток


101

Давайте напишем что-то, что учитывает информацию о том, что число больше или меньше нужного нам:

In [142]:
def game_core_v2(number):
    '''Сначала устанавливаем любое random число, а потом уменьшаем или увеличиваем его
        в зависимости от того, больше оно или меньше нужного.
        Функция принимает загаданное число и возвращает число попыток'''
    count = 1
    predict = np.random.randint(1,101)
    while number != predict:
        count+=1
        if number > predict: 
            predict += 1
        elif number < predict: 
            predict -= 1
    return(count) # выход из цикла, если угадали

In [143]:
# Проверяем
score_game(game_core_v2)

Ваш алгоритм угадывает число в среднем за 33 попыток


33

Попробуем улучшить алгоритм. Начнем с середины интервала а дальше перебором.

In [144]:
def game_new_core_v1(number):
    '''Начинаем с числа - половины диапозона, а потом уменьшаем или увеличиваем его
        в зависимости от того, больше оно или меньше нужного.
        Функция принимает загаданное число и возвращает число попыток'''
    count = 1 # счетчик попыток
    max_number = 100
    predict = int(max_number/2)
    while number != predict:
        count += 1
        if number > predict:
            predict += 1
        elif number < predict:
            predict -= 1
    return(count)

In [145]:
# Проверяем
score_game(game_new_core_v1)

Ваш алгоритм угадывает число в среднем за 25 попыток


25

Уже лучше, но не достаточно хорошо.
Попробуем реализовать метод бинарного поиска.
Используем дробление на половины интервалов поиска и смещая верхние и нижние границы.

In [146]:
def binary_search(number, lower_number=1, upper_number=100):
    '''Выбираем число методом половинного деления со смещением границы, 
        с учетом предыдущих максимальных и минимальных значений.
        Функция принимает загаданное число, [значения диапазона отгадывания] 
        и возвращает число попыток'''
    count = 1 # счетчик попыток
    predict = int((lower_number + upper_number)/2) # середина интервала

    #debug
    #print(f'cnt:{count}, prd:{predict}, lower:{lower_number}, upper:{upper_number}')

    while number != predict:
        
        if number > predict: lower_number = predict + 1 # сдвигаем вверх нижнюю границу
        elif number < predict: upper_number = predict - 1 # сдвигаем вниз верхнюю границу

        predict = int((lower_number + upper_number)/2) # середина интервала
        count += 1
        
        #debug
        #print(f'cnt:{count}, prd:{predict}, lower:{lower_number}, upper:{upper_number}')
            
    #debug        
    #print(count, predict)
    
    return(count)

In [147]:
def game_new_core_v4(number):
    '''Движок для определения загаданного числа Версия 4 (Бинарный поиск)
       Функция принимает загаданное число и возвращает число попыток'''
    count = binary_search(number)
    return(count)

In [148]:
# Проверяем
score_game(game_new_core_v4)

Ваш алгоритм угадывает число в среднем за 5 попыток


5

Круто!
А можно еще лучше?
В интрнетах пишут, что можно попробовать экспоненциальный поиск

In [153]:
def exponential_serch(number, start=1):
    '''Выбираем число методом перехода в экспоненциальные позиции (во вторую степень).
        Функция принимает загаданное число и стартовую позицию поиска. 
        Возвращает число попыток, нижний диапазон и верхний диапазон поиска, в котором лежит загаднное число,
        предсказываемое число и флаг угадано ли уже число'''
    count = 1 # счетчик попыток
    upper_number = start
    lower_number = 1
    predict = start
    predicted = False

    #debug
    #print(f'cnt:{count}, nbr:{number}, prd:{predict}, lower:{lower_number}, upper:{upper_number}')
    
    while number > predict:
        predict *= 2
        count += 1
        lower_number = upper_number
        upper_number = predict
      
        #debug
        #print(f'cnt:{count}, nbr:{number}, prd:{predict}, lower:{lower_number}, upper:{upper_number}')
    
    if predict == number: predicted = True

    return(count, lower_number, upper_number, predict, predicted)    

In [154]:
def game_new_core_v5(number):
    '''Движок для определения загаданного числа Версия 5 (Экспонентный поиск)
       Функция принимает загаданное число и возвращает число попыток'''
    count, lower_number, upper_number, predict, predicted = exponential_serch(number,64)
    
    if predicted: return(count)
    
    count += binary_search(number, lower_number, upper_number)
    
    return(count)

In [155]:
# Проверяем
score_game(game_new_core_v5)

Ваш алгоритм угадывает число в среднем за 6 попыток


6

Хорошо конечно, но не на таком маленьком диапазоне.
Собственно приходим к решению поставленной задачи.

# Решение
За самое меньше число попыток угадывает функция: serpent_core_v4.
Используется алгоритм бинарного поиска [Wikipedia](https://ru.wikipedia.org/wiki/%D0%94%D0%B2%D0%BE%D0%B8%D1%87%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D0%B8%D1%81%D0%BA)

In [156]:
score_game(game_new_core_v4)

Ваш алгоритм угадывает число в среднем за 5 попыток


5